Graph generation functions

In [1]:
using LightGraphs

invLogit(x) = 1./(1.+e.^-x)   

function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end

function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)

    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end

function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end

const MAX_ITER = 5000
const STOP_DIFF = 0.0001;

Optimization functions

In [2]:
# a update(Newton Raphson)


#y-1./(1.+e.^-(a+a_0))+(u' * L)' + rho*L*(L*a-b)   


function gradient2(a,a_0,u,L,rho,b,y)
#    grad = grad+ (y[i]-invLogit(a+a_0))+(u' * L)[0:t_0] + rho*(L*a-append!(b, zeros(t-t_o,1)))
    grad = y-invLogit(a+a_0)+(u' * L)' - rho*L*(L*a-b)
    return grad
end;



function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return -1*hess
end;





function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - pinv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    return a
end

newton (generic function with 1 method)

In [3]:


levels = 10
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. ;
g = randEdgeGen(g,1000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
numnewnodes = 5
for i in 1:numnewnodes
    g = addPrefNode(g,b)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-1+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end

t = 2^levels-1+numnewnodes
t_0 = 2^levels-1
u = zeros(t,1)+0.2;
rho = 1.1
lambda = 1.1
a_0 = -7

-7

In [4]:
a = zeros(t,numnewnodes)
b = zeros(t)
u = zeros(t,numnewnodes)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;

In [69]:
for i in 1:numnewnodes
	a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,b[1:length(A[i])],u[1:length(A[i]),i])
	u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-b[1:length(A[i])])
end

In [70]:
a

1028×5 Array{Float64,2}:
 0.0639411  0.0662768  1.15472  1.42531  0.0552211
 0.0644472  0.0673546  1.1554   1.4318   0.0550881
 0.0660387  0.0669865  1.16091  1.43101  0.0585648
 0.0670678  0.0649485  1.16007  1.44117  0.0539712
 0.064436   0.0652506  1.15398  1.43668  0.0576488
 0.0672255  0.0671763  1.17962  1.43626  0.0704629
 0.0715314  0.0647635  1.16148  1.44026  0.0570951
 0.0673322  0.0627811  1.15707  1.43911  0.0554731
 0.0677535  0.0655064  1.16476  1.45321  0.0559102
 0.0655853  0.0678055  1.15651  1.4432   0.0586776
 0.0639427  0.0641931  1.15072  1.43216  0.0568692
 0.0721378  0.0784358  1.17358  1.45318  0.0598724
 0.0662229  0.063489   1.16996  1.43607  0.0630973
 ⋮                                                
 0.0637203  0.0672502  1.14056  1.46752  0.0505469
 0.0606701  0.0598653  1.13251  1.72914  0.04898  
 0.0637364  0.0682017  1.14813  1.77536  0.0546687
 0.0660929  0.062589   1.16128  1.51601  0.0547095
 0.0641668  0.060786   1.15503  1.45461  0.0536125
 0.062

In [74]:
sum(b)    

0.0